In [3]:
from Data.RecSys2022 import RecSys2022URMType
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from run_algorithm import test_algorithm, run_algorithm

model_class = MultiThreadSLIM_SLIMElasticNetRecommender
model_params = {'topK': 25000, 'l1_ratio': 1.0, 'alpha': 2e-4, 'workers': 6}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)

model_class = RP3betaRecommender
model_params = {'topK': 80, 'alpha': 0.719514,
                'beta': 0.229898, 'implicit': True, 'normalize_similarity': True}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)


Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 779 ( 1.9%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
Fitting model MultiThreadSLIM_SLIMElasticNetRecommender...
Fit params: {'topK': 25000, 'l1_ratio': 1.0, 'alpha': 0.0002, 'workers': 6}


  5%|▌         | 1056/19630 [00:30<03:15, 95.11it/s]

Computing evaluation...


100%|█████████▉| 19624/19630 [01:02<00:00, 313.73it/s]


EvaluatorHoldout: Processed 40850 (100.0%) in 14.79 sec. Users per second: 2762
MAP@10: 0.0311005
Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 728 ( 1.7%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
Fitting model RP3betaRecommender...
Fit params: {'topK': 80, 'alpha': 0.719514, 'beta': 0.229898, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: Similarity column 19630 (100.0%), 4556.56 column/sec. Elapsed time 4.31 sec
Computing evaluation...
EvaluatorHoldout: Processed 40901 (100.0%) in 14.19 sec. Users per second: 2882
MAP@10: 0.0306672


In [5]:
from hybrid import Hybrid

model_class = Hybrid
model_params = {}

test_algorithm(model_class, model_params,
               urm_type=RecSys2022URMType.ONE_INTERACTED, dummies=False)

Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 732 ( 1.8%) Users that have less than 1 test interactions
Dataset loaded successfully with URM type: ONE_INTERACTED
RP3betaRecommender: Similarity column 19630 (100.0%), 4511.16 column/sec. Elapsed time 4.35 sec




100%|█████████▉| 19624/19630 [02:13<00:00, 146.74it/s]









































































































































































































































































































































































































































Fitting model Hybrid...
Fit params: {}
Computing evaluation...


EvaluatorHoldout: Processed 40897 (100.0%) in 16.23 sec. Users per second: 2520
MAP@10: 0.0299327


In [1]:
# importing this library so the seed stays the same
import os
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_user_wise
import Utils.not_random
from Data.RecSys2022 import RecSys2022, RecSys2022URMType
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
    split_train_in_two_percentage_user_wise)
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

# building the dataset
# later we can call the build method to get different URM
dataset = RecSys2022()
dataset.build(type=RecSys2022URMType.ONE_INTERACTED)

urm = dataset.get_urm()
icm = dataset.get_icm()

urm_train_validation, urm_test = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.8)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

recommender_class = Hybrid

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation,
                                           evaluator_test=evaluator_test)

hyperparameters_range_dictionary = {
    "epsilon": Real(0.00, 1.0),
}

recommender_input_args = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_train],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[urm_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)


output_folder_path = "result_experiments_4/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 50 
n_random_starts = n_cases
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

hyperparameterSearch.search(recommender_input_args,
                            recommender_input_args_last_test=recommender_input_args_last_test,
                            hyperparameter_search_space=hyperparameters_range_dictionary,
                            n_cases=n_cases,
                            n_random_starts=n_random_starts,
                            save_model="last",
                            output_folder_path=output_folder_path,  # Where to save the results
                            output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
                            metric_to_optimize=metric_to_optimize,
                            cutoff_to_optimize=cutoff_to_optimize,
                            )


Tensorflow is not available
Setting seed random library, os and numpy seed to 18
Unzipping dataset...
Loading interactions...
Loading features...
Loading target ids...
Cleaning up...
Building URM and ICM with criteria ONE_INTERACTED...
EvaluatorHoldout: Ignoring 1409 ( 3.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 773 ( 1.9%) Users that have less than 1 test interactions


NameError: name 'Hybrid' is not defined